In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns

In [14]:
fulltrain = pd.read_csv("train.csv")

In [15]:
fulltrain = fulltrain.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

In [16]:
fulltrain.head()

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,Sales_unit_this_week,Sales_this_week,Returns_unit_next_week,Returns_next_week,adjusted_demand
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3


In [18]:
#1311			
fulltrain[(fulltrain['Client_ID'] == 2202759) & (fulltrain['Product_ID'] == 30532) & (fulltrain['Route_ID'] == 2124)].head(10)

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,Sales_unit_this_week,Sales_this_week,Returns_unit_next_week,Returns_next_week,adjusted_demand
2835419,3,1311,1,2124,2202759,30532,33,256.74,0,0.0,33
13918638,4,1311,1,2124,2202759,30532,9,70.02,0,0.0,9
13928964,4,1311,4,2124,2202759,30532,23,178.94,0,0.0,23
24756999,5,1311,1,2124,2202759,30532,16,124.48,0,0.0,16
24767263,5,1311,4,2124,2202759,30532,16,124.48,0,0.0,16
35236176,6,1311,1,2124,2202759,30532,14,108.92,0,0.0,14
45534123,7,1311,1,2124,2202759,30532,29,225.62,0,0.0,29
56003867,8,1311,1,2124,2202759,30532,22,171.16,0,0.0,22
66416014,9,1311,1,2124,2202759,30532,36,280.08,0,0.0,36


In [20]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'adjusted_demand',
                                  'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                                  'Agencia_ID': 'Sales_Depot_ID',
                                  'Canal_ID': 'Sales_Channel_ID',
                                  'Ruta_SAK': 'Route_ID',
                                  'Cliente_ID': 'Client_ID',
                                  'Venta_uni_hoy': 'Sales_unit_this_week',
                                  'Venta_hoy': 'Sales_this_week',
                                  'Dev_uni_proxima': 'Returns_unit_next_week',
                                  'Dev_proxima': 'Returns_next_week',
                                  'Demanda_uni_equil': 'adjusted_demand',
                                  'Producto_ID': 'Product_ID'})



#set a unique id for each client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

In [28]:
test_stats = train[['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']].copy(deep=True)

test_stats.head()

,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID
0,1110,7,3301,15766
1,1110,7,3301,15766
2,1110,7,3301,15766
3,1110,7,3301,15766
4,1110,7,3301,15766


In [56]:
testagg = train.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

testagg

Sales_Depot_ID Sales_Channel_ID Route_ID Client_ID Product_ID  \
                                                              nunique   
0                 1110                7     3301     15766         43   
1                 1110                7     3301     22926          3   
2                 1110                7     3301     24080          4   
3                 1110                7     3301     24695          5   
4                 1110                7     3301     50379          4   
...                ...              ...      ...       ...        ...   
2236017          25759                1     5517   4602899          7   
2236018          25759                1     5517   4666803          4   
2236019          25759                1     5517   4707699          6   
2236020          25759                1     5517   4708097          1   
2236021          25759                1     5517   8188699          1   

        adjusted_demand                  
                   mean median min  max  
0              5.525000    4.0   1   23  
1             15.941176   18.0   1   30  
2             15.150000   11.5   2   38  
3             11.857143    5.5   1  100  
4             14.142857    6.5   1   45  
...                 ...    ...  ..  ...  
2236017        8.857143    4.0   2   40  
2236018       11.500000    2.5   1   40  
2236019        3.000000    3.0   1    6  
2236020        1.000000    1.0   1    1  
2236021        2.000000    2.0   2    2  

[2236022 rows x 9 columns]

In [58]:
testagg['Product_ID']['nunique']

0          43
1           3
2           4
3           5
4           4
           ..
2236017     7
2236018     4
2236019     6
2236020     1
2236021     1
Name: nunique, Length: 2236022, dtype: int64

In [59]:
del testagg

In [60]:
#create a dataframe of aggregate statistics for each client
testagg = train.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['Sales_Depot_ID'] = testagg['Sales_Depot_ID']
client_stats['Sales_Channel_ID'] = testagg['Sales_Channel_ID']
client_stats['Route_ID'] = testagg['Route_ID']
client_stats['Client_ID'] = testagg['Client_ID']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

#create a dataframe of aggregate statistics for each product
testagg = train.sort_values(by=['Product_ID']).groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['Sales_Depot_ID'] = testagg['Sales_Depot_ID']
product_stats['Sales_Channel_ID'] = testagg['Sales_Channel_ID']
product_stats['Route_ID'] = testagg['Route_ID']
product_stats['Product_ID'] = testagg['Product_ID']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

client_stats.head()

,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Products,adj_dem_mean,adj_dem_median,adj_dem_min,adj_dem_max
0,1110,7,3301,15766,43,5.52,4,1,23
1,1110,7,3301,22926,3,15.94,18,1,30
2,1110,7,3301,24080,4,15.15,11,2,38
3,1110,7,3301,24695,5,11.86,5,1,100
4,1110,7,3301,50379,4,14.14,6,1,45


In [65]:
client_stats[(client_stats['Sales_Depot_ID'] == 1110) & (client_stats['Sales_Channel_ID'] == 7) & (client_stats['Route_ID'] == 3301) & (client_stats['Client_ID'] == 15766)]['adj_dem_mean'].iloc[0]

np.float64(5.52)

In [66]:
train[(train['Sales_Depot_ID'] == 1110) & (train['Sales_Channel_ID'] == 7) & (train['Route_ID'] == 3301) & (train['Client_ID'] == 15766)]['adjusted_demand'].mean().round(2)

np.float64(5.52)

In [67]:
#verify client_stats was done as intended

for i in range(5):
    randID = np.random.choice(train['ID'].unique().tolist())

    SDID = train[train['ID'] == randID]['Sales_Depot_ID'].iloc[0]
    SCID = train[train['ID'] == randID]['Sales_Channel_ID'].iloc[0]
    RID = train[train['ID'] == randID]['Route_ID'].iloc[0]
    CID = train[train['ID'] == randID]['Client_ID'].iloc[0]

    assert(client_stats[(client_stats['Sales_Depot_ID'] == SDID) & (client_stats['Sales_Channel_ID'] == SCID) & (client_stats['Route_ID'] == RID) & (client_stats['Client_ID'] == CID)]['adj_dem_mean'].iloc[0] == train[(train['Sales_Depot_ID'] == SDID) & (train['Sales_Channel_ID'] == SCID) & (train['Route_ID'] == RID) & (train['Client_ID'] == CID)]['adjusted_demand'].mean().round(2))

In [68]:
#verify product_stats was done as intended

for i in range(5):
    randID = np.random.choice(train['ID'].unique().tolist())

    SDID = train[train['ID'] == randID]['Sales_Depot_ID'].iloc[0]
    SCID = train[train['ID'] == randID]['Sales_Channel_ID'].iloc[0]
    RID = train[train['ID'] == randID]['Route_ID'].iloc[0]
    PID = train[train['ID'] == randID]['Product_ID'].iloc[0]

    assert(product_stats[(product_stats['Sales_Depot_ID'] == SDID) & (product_stats['Sales_Channel_ID'] == SCID) & (product_stats['Route_ID'] == RID) & (product_stats['Product_ID'] == PID)]['adj_dem_mean'].iloc[0] == train[(train['Sales_Depot_ID'] == SDID) & (train['Sales_Channel_ID'] == SCID) & (train['Route_ID'] == RID) & (train['Product_ID'] == PID)]['adjusted_demand'].mean().round(2))

In [21]:
train.head(10)

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,adjusted_demand,ID
0,4,1110,7,3301,15766,325,1,0
1,4,1110,7,3301,15766,328,1,1
2,3,1110,7,3301,15766,1212,3,2
3,4,1110,7,3301,15766,1212,4,2
4,5,1110,7,3301,15766,1212,5,2
5,8,1110,7,3301,15766,1212,4,2
6,9,1110,7,3301,15766,1212,1,2
7,3,1110,7,3301,15766,1216,4,3
8,4,1110,7,3301,15766,1216,2,3
9,5,1110,7,3301,15766,1216,3,3


In [22]:
train['ID'].value_counts()

ID
5           7
25          7
12          7
39          7
38          7
           ..
27752012    1
27752014    1
27752015    1
27752001    1
27752034    1
Name: count, Length: 26396648, dtype: int64

In [23]:
test['ID'].value_counts()

ID
9513052     2
8198804     2
38          2
5634971     2
13396484    2
           ..
27751906    1
27751908    1
27751911    1
27751924    1
31          1
Name: count, Length: 6237461, dtype: int64

In [17]:
train[train['ID'] == 6830047]

,Week_num,Sales_Depot_ID,Route_ID,Client_ID,Product_ID,adjusted_demand,ID
18452466,3,1311,2124,2202759,30532,33,6830047
18452467,4,1311,2124,2202759,30532,9,6830047
18452468,4,1311,2124,2202759,30532,23,6830047
18452469,5,1311,2124,2202759,30532,16,6830047
18452470,5,1311,2124,2202759,30532,16,6830047
18452471,6,1311,2124,2202759,30532,14,6830047
18452472,7,1311,2124,2202759,30532,29,6830047
18452473,8,1311,2124,2202759,30532,22,6830047
18452474,9,1311,2124,2202759,30532,36,6830047
